In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def calculate_similarity_with_tf_idf():

    MIGRATED_QUERY_PATH = "../Query Processing/4_json_results/migrated_query_data.json"
    MIGRATED_QUERY_IDENTIFIER = "migrated"
    migrated_query_data = load_query_data(MIGRATED_QUERY_PATH)

    NEW_QUERY_PATH = "../Query Processing/4_json_results/new_query_data.json"
    NEW_QUERY_IDENTIFIER = "new"
    new_query_data = load_query_data(NEW_QUERY_PATH)

    return calculate_similarity(migrated_query_data, new_query_data)

In [6]:
res = calculate_similarity_with_tf_idf()

TypeError: can only concatenate list (not "str") to list

In [4]:
res.to_excel('./Results/similarity_report_tf_idf.xlsx', index=False)

NameError: name 'res' is not defined

##### `load_query_data`

In [3]:
def load_query_data(PATH):

    with open(PATH) as f:
        query_meta_data = json.load(f)
    return query_meta_data

##### `calculate_similarity`

In [4]:
def calculate_similarity(migrated_query_data, new_query_data):

    migrated_queries = extract_text_from_json(migrated_query_data)
    new_queries = extract_text_from_json(new_query_data)

    vectorizer = TfidfVectorizer()
    all_queries = migrated_queries + new_queries
    tf_idf_matrix = vectorizer.fit_transform(all_queries)

    similarity_matrix = cosine_similarity(tf_idf_matrix[:len(migrated_queries)], tf_idf_matrix[len(migrated_queries):])

    final_df = pd.DataFrame(
        [(old_key, new_key, similarity_matrix[i][j])
         for i, old_key in enumerate(migrated_query_data.keys())
         for j, new_key in enumerate(new_query_data.keys())],
        columns=["migrated_report_name", "new_report_name", "similarity"]
    )

    return final_df

##### `extract_text_from_json`

In [5]:
def extract_text_from_json(json_data):
    reports = []

    for query_key, query_value in json_data.items():
        text = " ".join(query_value.get("tables_cleansed", []) + query_value.get("columns_cleansed", []))
        reports.append(text)

    return reports